In [1]:
import tarfile
import nltk
import sys
import random
import numpy as np
import sklearn
from sklearn.model_selection import KFold, ShuffleSplit
import string
import re
from sklearn.model_selection import train_test_split
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer



In [2]:
# training/testing data files
polaritytar = tarfile.open("../Data/review_polarity.tar.gz", "r")
polaritytar.extractall('../Data/Polarity_Data')

In [3]:
### get all the lines from all the reviews ###

# lines from negative reviews
neglines = []
for nfilename in os.listdir('../Data/Polarity_Data/txt_sentoken/neg'):
    openFile = open(('../Data/Polarity_Data/txt_sentoken/neg/' + nfilename),"r")
    neglines = openFile.readlines()

# lines from positive reviews
poslines = []
for pfilename in os.listdir('../Data/Polarity_Data/txt_sentoken/pos'):
    openFile = open(('../Data/Polarity_Data/txt_sentoken/pos/' + pfilename),"r")
    poslines = openFile.readlines()
    

In [4]:
### tokenize each line based on whitespace ###

# tokens for positive reviews
poslines_tokens = []
for line in poslines:
    l = []
    for word in line.split():
        l.append(word)
    poslines_tokens.append(l)
# tokens for negative reviews
neglines_tokens = []
for line in neglines:
    l = []
    for word in line.split():
        l.append(word)
    neglines_tokens.append(l)

In [5]:
### helper function to remove non-alphanumeric characters and lowercase each token ###
def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())

    return cleaned_tokens

In [6]:
### clean up the tokens list ###
positive_cleaned_tokens = []
negative_cleaned_tokens = []

for tokens in poslines_tokens:
    positive_cleaned_tokens.append(remove_noise(tokens, stop_words))

for tokens in neglines_tokens:
    negative_cleaned_tokens.append(remove_noise(tokens, stop_words))

NameError: name 'stop_words' is not defined

In [502]:
def get_tweets_for_model(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tokens)

In [503]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens)

In [504]:
positive_dataset = [(tweet_dict,"Positive")
                     for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict,"Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset


In [505]:
np_dataset = np.array(dataset)

In [506]:
# use k-fold cross validation with k = 10 to train and test
kfold = KFold(10, True, 1)
mean_accuracy = 0

for train, test in kfold.split(np_dataset):
    classifier = NaiveBayesClassifier.train(np_dataset[train])
    mean_accuracy = mean_accuracy + classify.accuracy(classifier, np_dataset[test])
    
print("Accuracy is:", mean_accuracy/10)


Accuracy is: 0.8281818181818184
